In [71]:
import os 
import caer
import canaro
import cv2 as cv
import numpy as np
import gc

In [72]:
IMG_SIZE = (80,80)
channels = 1
char_path = '/Users/hetan2/Desktop/simpsons /archive/simpsons_dataset'

In [73]:
char_dict ={}
for char in os.listdir(char_path):
    char_dict[char] = len(os.listdir(os.path.join(char_path,char)))
char_dict = caer.sort_dict(char_dict, descending= True)
char_dict

[('homer_simpson', 2246),
 ('ned_flanders', 1454),
 ('moe_szyslak', 1452),
 ('lisa_simpson', 1354),
 ('bart_simpson', 1342),
 ('marge_simpson', 1291),
 ('krusty_the_clown', 1206),
 ('principal_skinner', 1194),
 ('charles_montgomery_burns', 1193),
 ('milhouse_van_houten', 1079),
 ('chief_wiggum', 986),
 ('abraham_grampa_simpson', 913),
 ('sideshow_bob', 877),
 ('apu_nahasapeemapetilon', 623),
 ('kent_brockman', 498),
 ('comic_book_guy', 469),
 ('edna_krabappel', 457),
 ('nelson_muntz', 358),
 ('lenny_leonard', 310),
 ('mayor_quimby', 246),
 ('waylon_smithers', 181),
 ('maggie_simpson', 128),
 ('groundskeeper_willie', 121),
 ('barney_gumble', 106),
 ('selma_bouvier', 103),
 ('carl_carlson', 98),
 ('ralph_wiggum', 89),
 ('patty_bouvier', 72),
 ('martin_prince', 71),
 ('professor_john_frink', 65),
 ('snake_jailbird', 55),
 ('cletus_spuckler', 47),
 ('rainier_wolfcastle', 45),
 ('simpsons_dataset', 42),
 ('agnes_skinner', 42),
 ('sideshow_mel', 40),
 ('otto_mann', 32),
 ('fat_tony', 27),
 (

In [74]:
characters = []
count = 0
for i in char_dict:
    characters.append(i[0])
    count+=1
    if count>=10:
        break
characters

['homer_simpson',
 'ned_flanders',
 'moe_szyslak',
 'lisa_simpson',
 'bart_simpson',
 'marge_simpson',
 'krusty_the_clown',
 'principal_skinner',
 'charles_montgomery_burns',
 'milhouse_van_houten']

In [75]:
# create training data
train = caer.preprocess_from_dir(char_path, characters, channels= channels, IMG_SIZE=IMG_SIZE, isShuffle= True)

[INFO] Could not find a file to load from. Generating the training data
----------------------------------------------
[INFO] At 1000 files
[INFO] At 2000 files
[INFO] 2246 files found in 0.002894163131713867s
[INFO] At 1000 files
[INFO] 1454 files found in 0.001605987548828125s
[INFO] At 1000 files
[INFO] 1452 files found in 0.0014469623565673828s
[INFO] At 1000 files
[INFO] 1354 files found in 0.0013129711151123047s
[INFO] At 1000 files
[INFO] 1342 files found in 0.0012941360473632812s
[INFO] At 1000 files
[INFO] 1291 files found in 0.0013229846954345703s
[INFO] At 1000 files
[INFO] 1206 files found in 0.001125335693359375s
[INFO] At 1000 files
[INFO] 1194 files found in 0.0011088848114013672s
[INFO] At 1000 files
[INFO] 1193 files found in 0.0011298656463623047s
[INFO] At 1000 files
[INFO] 1079 files found in 0.0010302066802978516s
----------------------------------------------
[INFO] 13811 files preprocessed! Took 0m 10s


In [76]:
len(train)

13811

In [77]:
featureSet, labels = caer.sep_train(train, IMG_SIZE=IMG_SIZE)

In [78]:
from tensorflow.keras.utils import to_categorical
#normalize the featureSet (0,1)
featureSet = caer.normalize(featureSet)
labels = to_categorical(labels, len(characters))

In [79]:
x_train, x_val, y_train, y_val = caer.train_val_split(featureSet, labels, val_ratio=.2)


In [80]:
del train
del featureSet
del labels
gc.collect()

0

In [81]:
BATCH_SIZE = 32
EPOCHS = 10

In [82]:
# image data generator
datd_gen = canaro.generators.imageDataGenerator()
train_gen = datd_gen.flow(x_train, y_train, batch_size =BATCH_SIZE)